In [2]:
import torch
import numpy as np
import helpers as help
import pandas as pd
username = input("Enter database name")
password = input("Enter password")
load = help.DataBaseLoader(user=username, password=password, schema='mimiciv_hosp')


df = load['patients']

df.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,None
2,10000068,F,19,2160,2008 - 2010,None
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,None


In [3]:
id_to_age = df.set_index("subject_id")["anchor_age"]
id_to_age

subject_id
10000032    52
10000048    23
10000068    19
10000084    72
10000102    27
            ..
19999828    46
19999829    28
19999840    58
19999914    49
19999987    57
Name: anchor_age, Length: 299712, dtype: int64

In [4]:
import pickle
icd_ids = pickle.load(open("data/mimiciv_time_irregular_icd/dataset_ids.set", "rb"))
# ages over 89 are all set to 91 (18, ... 89, 91), 73 ages in total
# indices 0 - 71 represent (19 ~ 89, 91) ages
# when entire vector is 0 --> age: 18

new = id_to_age[list(icd_ids)]

FileNotFoundError: [Errno 2] No such file or directory: 'data/mimiciv_time_irregular_icd/dataset_ids.set'

In [ ]:
pickle.dump(id_to_age, open("data/mimiciv_age_antibiotic/id_to_age" + ".dict", "wb"), -1)

In [ ]:
load = help.DataBaseLoader(user=username, password=password, schema='mimiciv_derived')


df = load['antibiotic']
df = df[["subject_id", "antibiotic"]].drop_duplicates()
print(len(df))
df.head()

330618


,subject_id,antibiotic
0,13637586,*NF* Cefoxitin Sodium
1,11249454,*NF* Ceftaroline
3,18132540,*NF* Ceftaroline
14,11847456,*NF* Ceftaroline
15,16614128,*NF* Cefuroxime


In [ ]:
sorted_antibiotic_freqs = df.groupby('antibiotic').count()['subject_id'].sort_values(ascending=False).to_frame()
sorted_antibiotic_freqs = sorted_antibiotic_freqs.iloc[:72]["subject_id"] # result is top 72 antibiotics and corresponding frequencies in order, ignore "subject_id" label
pickle.dump(sorted_antibiotic_freqs, open("data/mimiciv_age_antibiotic/sorted_antibiotic_freqs"+".series", "wb"), -1)
antibiotic_to_index = {}
for i, antibiotic in enumerate(sorted_antibiotic_freqs.index):
    antibiotic_to_index[antibiotic] = i
pickle.dump(antibiotic_to_index, open("data/mimiciv_age_antibiotic/antibiotic_to_index"+".dict", "wb"), -1)
antibiotic_to_index

{'Vancomycin': 0,
 'CefazoLIN': 1,
 'Ciprofloxacin HCl': 2,
 'CefePIME': 3,
 'MetRONIDAZOLE (FLagyl)': 4,
 'CeftriaXONE': 5,
 'Levofloxacin': 6,
 'Piperacillin-Tazobactam': 7,
 'Ciprofloxacin IV': 8,
 'CefTRIAXone': 9,
 'CeFAZolin': 10,
 'Azithromycin': 11,
 'MetroNIDAZOLE': 12,
 'Sulfameth/Trimethoprim DS': 13,
 'Ampicillin-Sulbactam': 14,
 'Clindamycin': 15,
 'Amoxicillin-Clavulanic Acid': 16,
 'Cephalexin': 17,
 'Meropenem': 18,
 'Doxycycline Hyclate': 19,
 'Vancomycin Oral Liquid': 20,
 'Sulfameth/Trimethoprim SS': 21,
 'Cefpodoxime Proxetil': 22,
 'Ampicillin Sodium': 23,
 'CefTAZidime': 24,
 'Mupirocin Ointment 2%': 25,
 'Azithromycin ': 26,
 'Linezolid': 27,
 'Gentamicin Sulfate': 28,
 'Gentamicin': 29,
 'Nitrofurantoin Monohyd (MacroBID)': 30,
 'Piperacillin-Tazobactam Na': 31,
 'Nafcillin': 32,
 'Amoxicillin': 33,
 'Mupirocin Nasal Ointment 2%': 34,
 'Erythromycin': 35,
 'Aztreonam': 36,
 'Tobramycin Sulfate': 37,
 'Sulfameth/Trimethoprim Suspension': 38,
 'Penicillin G Potass

In [7]:
from tqdm import tqdm
id_to_antibiotic = df.set_index("subject_id")["antibiotic"]
id_to_antibiotic_indices = {}

for i, id in enumerate(tqdm(df['subject_id'].unique())):
    id_to_antibiotic_indices[id] = []
    for j, antibiotic in enumerate(id_to_antibiotic[id]):
        if antibiotic in antibiotic_to_index: # if not the first 72 most frequent antibiotic, ignore
            id_to_antibiotic_indices[id].append(antibiotic_to_index[antibiotic])
    id_to_antibiotic_indices[id] = np.unique(np.array(id_to_antibiotic_indices[id]))
    

pickle.dump(id_to_antibiotic_indices, open("data/mimiciv_age_antibiotic/id_to_antibiotic_indices"+".dict", "wb"), -1)

id_to_antibiotic_indices


100%|██████████| 97491/97491 [00:11<00:00, 8335.86it/s] 


{13637586: array([ 4,  5, 11, 26]),
 11249454: array([0, 1, 2, 7]),
 18132540: array([ 0,  2,  4,  6,  8, 18, 29, 44]),
 11847456: array([ 0,  3,  7,  8, 18, 20, 27, 28, 36, 56]),
 16614128: array([ 0,  1,  2,  3,  5, 16, 24, 30]),
 16089740: array([ 0,  3,  5,  8, 23, 27, 43, 56, 64]),
 17737450: array([ 0,  3,  7, 13, 27, 44, 56, 64]),
 19229277: array([ 0,  3,  5,  6,  7, 13, 37, 43, 44, 64]),
 10215503: array([ 0,  5, 23, 44, 60]),
 11148709: array([ 0,  6, 13, 15, 17, 25, 27, 60]),
 12373117: array([ 0,  3,  4,  5,  6, 18, 20, 21, 31, 34, 41, 54, 56, 60]),
 15252880: array([ 0,  1,  2,  3,  4,  5,  6,  8, 18, 32]),
 15202542: array([ 0,  3,  5,  6,  8, 36, 37, 60]),
 13478814: array([ 0,  2,  4,  7,  8,  9, 12, 13, 18, 20, 21, 27, 30, 36, 56, 60]),
 14078147: array([ 0, 43]),
 10346149: array([23, 33]),
 19714547: array([ 0,  3,  5,  7, 12, 14, 16, 17, 18, 19, 21, 32, 33, 38, 43]),
 18201522: array([11, 23, 33, 35]),
 17398386: array([ 0,  2,  4,  5, 14, 19, 20, 21, 33, 38]),
 162